# Packages

In [1]:
import requests
import json
import pandas as pd
import numpy as np
import glob
import datetime
import warnings
# ignore the warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import math
import seaborn as sns
import plotly.express as px
import ipywidgets as widgets
from ipywidgets import Layout
from ipywidgets import interact
import matplotlib.pyplot as plt
import plotly.graph_objects as go

import matplotlib.image as mpimg
import matplotlib.cm as cm
import matplotlib.widgets as mpl_widgets

from calendar import monthrange

from IPython.display import display
from time import sleep

# Login

In [2]:
# store login data in login.py
%run login.py

In [3]:
loginquery = f"""
mutation {{
  logIn(
      email:\"{login}\",
      password:\"{pwd}\") {{
    jwt {{
      token
      exp
    }}
  }}
}}
"""

In [4]:
url = 'https://api.numina.co/graphql'
mylogin = requests.post(url, json={'query': loginquery})
mylogin

<Response [200]>

In [5]:
token = mylogin.json()['data']['logIn']['jwt']['token']

In [6]:
expdate = mylogin.json()['data']['logIn']['jwt']['exp']
expdate

'2020-03-16T23:09:47.578415'

# Question 2

- Streetscape:
    - Event day: Everywhere, but they tend to use the stairs and go along the path around the area
    - No event day: Concentrated at the tables
- Under raincoat:
    - Event day: Tend to use the seatings more to have a rest and linger around the path
    - No event day: Just pass the chairs 
- Outside:
    - Event day: Sometimes it do not have any people outside(?), or they linger everywhere and seatings
    - No event day: Just go inside the entrance and exit

In [109]:
# define the locations
locations = {"Streetscape": "SWLSANDBOX1", 
             "Under Raincoat": "SWLSANDBOX2", 
             "Outside": "SWLSANDBOX3"}

# read the counts data according to the parameters
def read_count(locations, str_start, str_end, interval):
    all_counts = pd.DataFrame(columns=["time", "Streetscape", 
                                       "Under Raincoat", "Outside"])
    # loop over the locations to get the different result
    for loc in locations:
        countquery = f"""
        query{{
          feedCountMetrics(
            serialnos:\"{locations[loc]}\",
            startTime:\"{str_start}\",
            endTime:\"{str_end}\",
            objClasses:["pedestrian"],
            timezone:"America/New_York",
            interval:\"{interval}\") {{
            edges {{
              node {{
                serialno
                result
                objClass
                time
              }}
            }}
          }}
        }}
        """
        counts = requests.post(url, json={'query': countquery}, 
                               headers = {'Authorization':token})
        counts = counts.json()['data']['feedCountMetrics']['edges']
        counts_lst = []
        for i in range(len(counts)):
            counts_lst += [counts[i]['node']]
        counts = pd.DataFrame.from_dict(
            counts_lst, orient='columns').drop(
            ['objClass','serialno'],axis=1).rename(columns={'result':str(loc)})
        # create new dataframe or fill in na
        if all_counts['time'].count() == 0:
            all_counts = all_counts.append(counts)
        else:
            all_counts = all_counts.combine_first(counts)
        # convert the number of people to float
        all_counts.iloc[:,:3] = all_counts.iloc[:,:3].astype(float)
        # sum all data together to get the overall information
        all_counts['total'] = all_counts.iloc[:,:3].sum(1)
        # change the time to timestamp
        all_counts['time'] = pd.to_datetime(
            all_counts['time'], format="%Y-%m-%dT%H:%M:%S%z")
    return all_counts

In [206]:
# find a way to define an event
# we might define the event as larger than 500 people overall?
def check_events(count_data):
    sub_count = read_count(locations,datetime.datetime.strftime(
                        count_data.iloc[0,3],'%Y-%m-%dT%H:%M:%S'),
                               datetime.datetime.strftime(
                        count_data.iloc[-1,3],
                                   '%Y-%m-%dT%H:%M:%S'),"1h")
    sub_count['event'] = sub_count['total'].apply(lambda x: 1 if x>=500 else 0)
    sub_count['time'] = sub_count['time'].apply(lambda x: str(x).split(' ')[0]+\
                                                " 00:00:00-05:00:00")
    sub_count = sub_count.groupby('time').max().reset_index()
    sub_count['time'] = pd.to_datetime(
            sub_count['time'], format="%Y-%m-%d %H:%M:%S%z")
    count_data = count_data.merge(sub_count[['time','event']],on='time')
    
    return count_data

In [28]:
def read_heatmap(location, str_start, str_end):
    heatmapquery = f"""
    query {{
      feedHeatmaps(
        serialno:\"{locations[location]}\",
        startTime:\"{str_start}\",
        endTime:\"{str_end}\",
        objClasses:["pedestrian"],
        timezone:"America/New_York") {{
        edges {{
          node {{
            time
            objClass
            heatmap
          }}
        }}
      }}
    }}
    """

    heatmap_request = requests.post(url, json={'query': heatmapquery}, 
                                    headers = {'Authorization':token})
    heatmaps = heatmap_request.json()["data"]["feedHeatmaps"]["edges"][0]["node"]["heatmap"]
    heatmaps = pd.DataFrame(heatmaps, columns=["x", "y", "value"])
    heatmaps = heatmaps.append(pd.DataFrame(
            {"x": [0, 0], "y": [0, 0], "value": [0, 1]}))
    return heatmaps

In [209]:
from plotly.subplots import make_subplots

# create dataframe for question 2
dates = [datetime.date(2019, 2, j) for j in range(20, 29)]
dates += [datetime.date(i, j, 1) for i in range(2019, 2020) for j in range(3, 13)]
dates += [datetime.date(2020, 1, i) for i in range(1, 13)]
hour = [str(i).zfill(2)+":"+j+":00" for i in range(0,24) for j in ["00","15","30","45"]]
all_events = check_events(read_count(
        locations,"2019-02-20T00:00:00","2020-01-12T00:00:00","24h"))
events_option = all_events[all_events['event']==1]['time'].astype(str).str[:10]
non_events_option = all_events[all_events['event']==0]['time'].astype(str).str[:10]

In [210]:
all_events

,Outside,Streetscape,Under Raincoat,time,total,event
0,0.0,3272.0,0.0,2019-02-20 00:00:00-05:00,3272.0,1
1,0.0,158.0,0.0,2019-02-21 00:00:00-05:00,158.0,0
2,0.0,83.0,0.0,2019-02-22 00:00:00-05:00,83.0,0
3,0.0,8.0,0.0,2019-02-23 00:00:00-05:00,8.0,0
4,0.0,6.0,0.0,2019-02-24 00:00:00-05:00,6.0,0
...,...,...,...,...,...,...
320,242.0,1165.0,0.0,2020-01-06 00:00:00-05:00,1407.0,0
321,820.0,305.0,0.0,2020-01-07 00:00:00-05:00,1125.0,0
322,80.0,230.0,0.0,2020-01-08 00:00:00-05:00,310.0,0
323,109.0,619.0,0.0,2020-01-09 00:00:00-05:00,728.0,0


In [223]:
# define the function to complete question 2
def compare_events(date, event, non_event, location, interval, time):
    # calculate the time
    time_event_asdate = datetime.datetime.strptime(event+" "+time,
                    "%Y-%m-%d %H:%M:%S")
    time_non_event_asdate = datetime.datetime.strptime(non_event+" "+time,
                    "%Y-%m-%d %H:%M:%S")
    if interval == '15 min':
        time_end_event = str(time_event_asdate+datetime.timedelta(minutes=15))
        time_non_end_event = str(time_non_event_asdate+datetime.timedelta(minutes=15))
    elif interval == 'hour':
        time_end_event = str(time_event_asdate+datetime.timedelta(hours = 1))
        time_non_end_event = str(time_non_event_asdate+datetime.timedelta(hours=1))
    elif interval == '6 hours':
        time_end_event = str(time_event_asdate+datetime.timedelta(hours = 6))
        time_non_end_event = str(time_non_event_asdate+datetime.timedelta(hours=6))
    else:
        time_end_event = str(time_event_asdate+datetime.timedelta(days = 1))
        time_non_end_event = str(time_non_event_asdate+datetime.timedelta(days=1))
    time_end_event = time_end_event.replace(" ","T")
    time_non_end_event = time_non_end_event.replace(" ","T")
    
    # create heatmaps according to the event or non event day
    heatmaps_event = read_heatmap(location, 
                    datetime.datetime.strftime(
                        time_event_asdate,'%Y-%m-%dT%H:%M:%S'), time_end_event)
    heatmaps_non_event = read_heatmap(location, 
                    datetime.datetime.strftime(
                        time_non_event_asdate,'%Y-%m-%dT%H:%M:%S'), time_non_end_event)
    # plot heatmaps
    fig,axs = plt.subplots(1,2,figsize=(20,7),sharey=True)
    ax = axs[0].scatter(x=heatmaps_event["x"], y=heatmaps_event["y"],
                            c=heatmaps_event["value"],cmap="jet",vmin=0, vmax=1)
    axs[0].set_title("Event Day "+str(event),fontsize=15)
    axs[1].scatter(x=heatmaps_non_event["x"], y=heatmaps_non_event["y"],
                   c= heatmaps_non_event["value"],cmap="jet",vmin=0, vmax=1)
    axs[1].set_title("No Event Day "+str(non_event),fontsize=15)
    for i in range(0,2):
        if location == 'Streetscape':
            img = plt.imread("streetscape.png")
        elif location == 'Under Raincoat':
            img = plt.imread("under_raincoat.png")
        else:
            img = plt.imread("outside.png")
        axs[i].imshow(img,zorder=0)
        axs[i].set_yticks([])
        axs[i].set_xticks([])
        axs[i].axis('off')
    # color bar
    cbar = fig.colorbar(ax, ax=axs.ravel().tolist())
    fig.suptitle('Heatmap for '+location,fontsize=20)
    plt.show()


# interactive widget
date_widget = widgets.SelectionRangeSlider(
             options=dates,
             index=(0, 7),
             description='Date:',
             disabled=False, layout=Layout(width='45%'))
event_widget = widgets.Dropdown(
             options=events_option,
             description='Event:',
             disabled=False, layout=Layout(width='45%'))
non_event_widget = widgets.Dropdown(
             options=non_events_option,
             description='No Event:',
             disabled=False, layout=Layout(width='45%'))
# update interactive widget
def update_event_day(*args):
    # define the start and the end period
    start = datetime.datetime.combine(date_widget.value[0], 
                        datetime.datetime.min.time())
    end = datetime.datetime.combine(date_widget.value[1],
                        datetime.datetime.min.time())
    event_widget.options = [time for time in events_option if \
                            ((datetime.datetime.strptime(time,
                    "%Y-%m-%d") >= start) & (datetime.datetime.strptime(time,
                    "%Y-%m-%d") <= end))]
    
    non_event_widget.options = [time for time in non_events_option if\
                                ((datetime.datetime.strptime(time,
                    "%Y-%m-%d") >= start) & (datetime.datetime.strptime(time,
                    "%Y-%m-%d") <= end))]
    
date_widget.observe(update_event_day, 'value')

from ipywidgets import interactive,HBox,VBox
# present the dashboard
widget = interactive(compare_events, 
         date = date_widget,
         event = event_widget, 
         non_event = non_event_widget, #change to non_event
         location = widgets.Dropdown(
             options=['Streetscape','Under Raincoat','Outside'],
             description='Location:',
             disabled=False, layout=Layout(width='45%')), 
         interval = widgets.Dropdown(
             options=['15 min','hour','6 hours','day'],
             description='By:',
             disabled=False, layout=Layout(width='45%')), 
         time = widgets.Dropdown(
             options=hour,
             description='Time:',
             disabled=False, layout=Layout(width='45%')), )

controls = HBox(widget.children[1:-1], 
                layout = Layout(flex_flow='row wrap'))
output = widget.children[-1]
display(VBox([widget.children[0], controls, output]))